In [1]:
import torch
from utils.sh_utils import RGB2SH
import geo.mesh_utils as mesh_utils
from utils.general_utils import inverse_sigmoid
from scene.gaussian_geo_model_mlp_flex import GaussianGeoModel
from geo.flexicubes import FlexiCubes


In [2]:
self = GaussianGeoModel(sh_degree=3)
debug_dict = torch.load('debug.pth')
saved_attr = ['verts', 'max_displacement', 'deform', 'sdf', 'indices', 'grid_res', 'per_cube_weights', 'bc_coords', 'rad_base', 'spatial_lr_scale', 'max_sh_degree',]
for k in saved_attr:
    self.__setattr__(k, debug_dict[k])
self.create_from_pcd(None, debug_dict['spatial_lr_scale'])
self.mlp_texture.load_state_dict(debug_dict['mlp_texture'])
self.marching_geo    = FlexiCubes()

warnning!! using mlp texture
Encoder output: 32 dims


In [3]:
# from torch.profiler import profile, record_function, ProfilerActivity

# with profile(activities=[ProfilerActivity.CUDA], with_stack=True) as prof:
#     with record_function("getMesh"):
#         pass
starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
from contextlib import contextmanager

@contextmanager
def count_time(name):
    starter.record()
    yield
    ender.record(); torch.cuda.synchronize()
    print(f"{name}:\t{starter.elapsed_time(ender):>6.2f}ms")


In [5]:
device = 'cuda'

with count_time('Get_Mesh'):
    verts, faces = self.getMesh()

with count_time('Mesh_to_GS'):
    # triangle space definition & data shape
    face_normals = mesh_utils.face_normals(verts, faces, unit=True)
    face_x = torch.nn.functional.normalize(verts[faces[:, 1]] - verts[faces[:, 0]])
    face_y = torch.nn.functional.normalize(torch.cross(face_normals, face_x, dim=-1))
    self.rot_t2w = torch.stack([face_x, face_y, face_normals], dim=2)
    n_face = faces.shape[0]
    N_gs_per_face = self.bc_coords.shape[0]
    N_gs = n_face * N_gs_per_face

    # gaussian means
    face_vert = verts[faces]
    gs_xyz = torch.matmul(
        self.bc_coords.view(1, -1, 3), face_vert)  # [1, n_inner_gs, 3] @ [n_triangle, 3, 3]
    gs_xyz = gs_xyz.view(-1, 3)  # [n_triangle*n_inner_gs, 3]

with count_time('compute_Cov3D-0'):
    # gaussian covariance (triangle space)
    v1v2_xlen = torch.norm(face_vert[:, 1, :] - face_vert[:, 0, :], dim=-1)
    v1v2_xlen = v1v2_xlen.clamp_min(1e-10)  # there are `point` triangles
    v1v3 = face_vert[:, 2, :] - face_vert[:, 0, :]

    # affine matrix M: from equilateral to current triangle, ME = A 
    # 1. no shift, no scale/direction change at x-axis, so
    # M must be [[1, m1], [0, m2]], with m1, m2 to be solved.
    # 2. assume only the third pair of points E_x/y, A_x/y change,
    # m1 = (A_x-E_x)/E_y, m2 = A_y/E_y
    A = torch.stack([(v1v3*face_x).sum(-1), (v1v3*face_y).sum(-1)], dim=-1)
    E = torch.stack([v1v2_xlen/2., v1v2_xlen/2. * (3**0.5)], dim=-1)
    M_2d = torch.zeros((n_face, 2, 2), device=device)
    M_2d[:, 0, 0] = 1.
    M_2d[:, 0, 1] = (A[:, 0] - E[:, 0]) / E[:, 1]
    M_2d[:, 1, 1] = A[:, 1] / E[:, 1]

with count_time('compute_Cov3D-3'):
    # L=M R_init S, 2D triangle space, here R_init=I
    s_scalar = (v1v2_xlen * self.rad_base)  # s_scalar is std-deviation
    L = M_2d * s_scalar.view(-1, 1, 1)  # [n_face, 2, 2] * [n_tri, 1, 1]

    cov3D_L = torch.zeros((n_face, N_gs_per_face, 3, 3), device=device)
    cov3D_L[:, :, :2, :2] = L.view(n_face, 1, 2, 2)
    cov3D_L[:, :, 2, 2] = self.spatial_lr_scale * 1e-6
    cov3D_L = cov3D_L.view(-1, 3, 3)

with count_time("SH_query"):
    # gaussian colors
    features = torch.zeros((N_gs, 3, (self.max_sh_degree + 1) ** 2), device=device)
    features[:, :3, 0 ] = RGB2SH(self.mlp_texture.sample(gs_xyz))
    # gaussian opacity (triangle space)
    opacities = inverse_sigmoid(0.9999 * torch.ones((N_gs, 1), device=device))
with count_time("Post_Process"):
    # -------------------------------------------------- #
    # construct parameters and config optimizer
    # -------------------------------------------------- #
    self._xyz = gs_xyz
    self._features_dc = features[:,:,0:1].transpose(1, 2).contiguous()  # nn.Parameter(features[:,:,0:1].transpose(1, 2).contiguous().requires_grad_(True))
    self._features_rest = features[:,:,1:].transpose(1, 2).contiguous()  # nn.Parameter(features[:,:,1:].transpose(1, 2).contiguous().requires_grad_(True))
    self._scaling = None  # nn.Parameter(scale2D.requires_grad_(True))
    self._rotation = None  # nn.Parameter(rots2D.requires_grad_(True))
    self.cov3D_L = cov3D_L
    self._opacity = opacities  # nn.Parameter(opacities.requires_grad_(True))
    self.max_radii2D = torch.zeros((gs_xyz.shape[0]), device=device)


Get_Mesh:	  8.29ms
Mesh_to_GS:	  1.25ms
compute_Cov3D-0:	  0.28ms
compute_Cov3D-3:	  0.21ms
SH_query:	  7.44ms
Post_Process:	  1.19ms


In [20]:
from utils.general_utils import strip_symmetric

with count_time("before-render"):
    n_triangle = self.rot_t2w.shape[0]
    n_inner_gs = self.bc_coords.shape[0]

    L = self.cov3D_L.view(n_triangle, n_inner_gs, 3, 3)[:, 0, :, :].view(n_triangle, 1, 3, 3)

    # world space R
    R_t2w = self.rot_t2w.view(n_triangle, 1, 3, 3)
    L = torch.matmul(R_t2w, L).view(-1, 3, 3)  # [n_triangle*n_inner_gs,3,3]

    actual_covariance = L @ L.transpose(1, 2)
    symm = strip_symmetric(actual_covariance)

    symm.view(n_triangle, 1, 6).expand(-1, n_inner_gs, 6).contiguous()


before-render:	  1.25ms


In [14]:
symm.shape

torch.Size([591168, 6])

In [8]:
from utils.sh_utils import eval_sh
import torch

deg = 3
sh = torch.zeros((400, 400, 3, 16))
dirs = torch.zeros((400, 400, 3))
result = eval_sh(deg, sh, dirs)
print(result.min())

tensor(0.)
